- hey Brian sorry i’m having a little trouble locating individual barcode bams.  for the bedgraph of individual cells (like the top 100 cells used in the browser views in figs 5-7), the input bams would be in which directories?  i need to just glance at a handful of those with lenticommon mapped.

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tnrange, tqdm_notebook
pd.set_option('max_colwidth', 400)

In [2]:
barcodes_file = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/inputs/Top_100_apo_rps2_barcodes.xlsx'

bam_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/bam_files'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/tmp/top100_bams_for_kris'

In [3]:
apo_barcodes = pd.read_excel(barcodes_file, sheet_name='Sheet1')
apo_barcodes.sort_values(by=['n_edits'], ascending=False, inplace=True)
apo_barcodes.reset_index(drop=True, inplace=True)
apo_barcodes['rank'] = apo_barcodes.index + 1

rps2_barcodes = pd.read_excel(barcodes_file, sheet_name='Sheet2')
rps2_barcodes.sort_values(by=['n_edits'], ascending=False, inplace=True)
rps2_barcodes.reset_index(drop=True, inplace=True)
rps2_barcodes['rank'] = rps2_barcodes.index + 1

rps2_barcodes.head()

,barcode,class,n_edits,batch,rank
0,RPS2_possorted_genome_bam_MD-CGGCAGTGTGCGTGCT-1-RPS2-Edits,RPS2-Edits,5.986034,RPS2-fSTAMP,1
1,RPS2_possorted_genome_bam_MD-ACTGTGATCAGACCGC-1-RPS2-Edits,RPS2-Edits,4.031267,RPS2-fSTAMP,2
2,RPS2_possorted_genome_bam_MD-GACCCAGCACTTCAAG-1-RPS2-Edits,RPS2-Edits,3.897882,RPS2-fSTAMP,3
3,RPS2_possorted_genome_bam_MD-CTCATCGGTCGAGCAA-1-RPS2-Edits,RPS2-Edits,3.826919,RPS2-fSTAMP,4
4,RPS2_possorted_genome_bam_MD-ACAGCCGAGCCTGGAA-1-RPS2-Edits,RPS2-Edits,3.793303,RPS2-fSTAMP,5


In [4]:
def get_bam_from_barcode_index(row, input_dir=bam_dir):
    barcode = row['barcode']
    bam = os.path.join(input_dir, '{}.bam'.format(barcode.replace('-Apo-Edits','').replace('-RPS2-Edits','')))
    assert os.path.exists(bam)
    return bam

In [5]:
apo_barcodes['bam'] = apo_barcodes.apply(get_bam_from_barcode_index, axis=1)
rps2_barcodes['bam'] = rps2_barcodes.apply(get_bam_from_barcode_index, axis=1)

apo_barcodes.head()

,barcode,class,n_edits,batch,rank,bam
0,Apo_Control_possorted_genome_bam_MD-AGAAGTACACGCGGTT-1-Apo-Edits,Apo-Edits,1.298744,RPS2-fSTAMP,1,/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/bam_files/Apo_Control_possorted_genome_bam_MD-AGAAGTACACGCGGTT-1.bam
1,Apo_Control_possorted_genome_bam_MD-CATACAGCAACCAACT-1-Apo-Edits,Apo-Edits,0.815492,RPS2-fSTAMP,2,/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/bam_files/Apo_Control_possorted_genome_bam_MD-CATACAGCAACCAACT-1.bam
2,Apo_Control_possorted_genome_bam_MD-GGTGTCGGTCTGCCTT-1-Apo-Edits,Apo-Edits,0.720313,control-oSTAMP,3,/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/bam_files/Apo_Control_possorted_genome_bam_MD-GGTGTCGGTCTGCCTT-1.bam
3,Apo_Control_possorted_genome_bam_MD-TAACTTCAGCCAAGGT-1-Apo-Edits,Apo-Edits,0.565517,control-oSTAMP,4,/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/bam_files/Apo_Control_possorted_genome_bam_MD-TAACTTCAGCCAAGGT-1.bam
4,Apo_Control_possorted_genome_bam_MD-CTCAACCCAGACGGAT-1-Apo-Edits,Apo-Edits,0.563772,control-oSTAMP,5,/home/bay001/projects/kris_apobec_20200121/permanent_data2/03_scRNA_APOBEC_RPS2/bam_files/Apo_Control_possorted_genome_bam_MD-CTCAACCCAGACGGAT-1.bam


### Now copy the bams over to a less insane place for Kris

In [6]:
progress = tnrange(len(apo_barcodes['bam']))
for bam in apo_barcodes['bam']:
    cmd = 'cp {} {};samtools index {}'.format(bam, os.path.join(output_dir, os.path.basename(bam)), os.path.join(output_dir, os.path.basename(bam)))
    ! $cmd
    progress.update(1)
    
progress = tnrange(len(rps2_barcodes['bam']))
for bam in rps2_barcodes['bam']:
    cmd = 'cp {} {};samtools index {}'.format(bam, os.path.join(output_dir, os.path.basename(bam)), os.path.join(output_dir, os.path.basename(bam)))
    ! $cmd
    progress.update(1)